# import

In [13]:
import sys
import sys
import os

# Get the current working directory (notebooks do not have __file__)
data_path = os.path.join(os.getcwd(), 'data')
sys.path.append(data_path)


#Use this one if you are running the code from a script not a notebook
# import sys
# import os

# # Add the 'data' directory to the Python path
# data_path = os.path.join(os.path.dirname(os.path.abspath(__file__)), 'data')
# sys.path.append(data_path)

In [14]:
from model.bert import bert_ATE, bert_ABSA

from dataset import dataset_ATM, dataset_ABSA

In [15]:
from torch.utils.data import DataLoader, ConcatDataset
from transformers import BertTokenizer
import torch
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import time
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from transformers import AdamW

In [16]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#pretrain_model_name = "bert-base-uncased"      # I will try absa 
pretrain_model_name = "bert-large-uncased"
tokenizer = BertTokenizer.from_pretrained(pretrain_model_name)
lr = 2e-5      #Learning rate
model_ATE = bert_ATE(pretrain_model_name).to(DEVICE)
optimizer_ATE = torch.optim.Adam(model_ATE.parameters(), lr=lr)
#model_ABSA = bert_ABSA(pretrain_model_name).to(DEVICE)
model_ABSA = bert_ABSA(pretrain_model_name, DEVICE).to(DEVICE)
#optimizer_ABSA = torch.optim.Adam(model_ABSA.parameters(), lr=lr)
optimizer_ABSA = AdamW(
    model_ABSA.parameters(), 
    lr=lr, 
    weight_decay=1e-4  # or 1e-5
)

In [17]:
def evl_time(t):
    min, sec= divmod(t, 60)
    hr, min = divmod(min, 60)
    return int(hr), int(min), int(sec)

def load_model(model, path):
    model.load_state_dict(torch.load(path), strict=False)
    return model
    
def save_model(model, name):
    torch.save(model.state_dict(), name)

# Acpect Term Extraction

In [20]:

restaurants_train_ds = dataset_ATM(pd.read_csv("data/restaurants_train.csv"), tokenizer)
restaurants_test_ds = dataset_ATM(pd.read_csv("data/restaurants_test.csv"), tokenizer)


In [21]:
# train_ds = ConcatDataset([restaurants_train_ds])
# test_ds = ConcatDataset([restaurants_test_ds])

train_ds =restaurants_train_ds
test_ds = restaurants_test_ds

In [22]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors = pad_sequence(ids_tensors, batch_first=True)

    tags_tensors = [s[2] for s in samples]
    tags_tensors = pad_sequence(tags_tensors, batch_first=True)

    pols_tensors = [s[3] for s in samples]
    pols_tensors = pad_sequence(pols_tensors, batch_first=True)
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1)
    
    return ids_tensors, tags_tensors, pols_tensors, masks_tensors

In [23]:
train_loader = DataLoader(train_ds, batch_size=5, collate_fn=create_mini_batch, shuffle = True)
test_loader = DataLoader(test_ds, batch_size=50, collate_fn=create_mini_batch, shuffle = True)

In [10]:
# for batch in train_loader:
#     w,x,y,z = batch
#     print(w)
#     print(w.size())
#     print(x)
#     print(x.size())
#     print(y)
#     print(y.size())
#     print(z)
#     print(z.size())
#     break

In [24]:
def train_model_ATE(loader, epochs):
    all_data = len(loader)
    for epoch in range(epochs):
        finish_data = 0
        losses = []
        current_times = []
        correct_predictions = 0
        
        for data in loader:
            t0 = time.time()
            ids_tensors, tags_tensors, _, masks_tensors = data
            ids_tensors = ids_tensors.to(DEVICE)
            tags_tensors = tags_tensors.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            loss = model_ATE(ids_tensors=ids_tensors, tags_tensors=tags_tensors, masks_tensors=masks_tensors)
            losses.append(loss.item())
            loss.backward()
            optimizer_ATE.step()
            optimizer_ATE.zero_grad()

            finish_data += 1
            current_times.append(round(time.time()-t0,3))
            current = np.mean(current_times)
            hr, min, sec = evl_time(current*(all_data-finish_data) + current*all_data*(epochs-epoch-1))
            print('epoch:', epoch, " batch:", finish_data, "/" , all_data, " loss:", np.mean(losses), " hr:", hr, " min:", min," sec:", sec)         

        save_model(model_ATE, 'bert_ATE2.pkl')
        
def test_model_ATE(loader):
    pred = []
    trueth = []
    with torch.no_grad():
        for data in loader:

            ids_tensors, tags_tensors, _, masks_tensors = data
            ids_tensors = ids_tensors.to(DEVICE)
            tags_tensors = tags_tensors.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            outputs = model_ATE(ids_tensors=ids_tensors, tags_tensors=None, masks_tensors=masks_tensors)

            _, predictions = torch.max(outputs, dim=2)

            pred += list([int(j) for i in predictions for j in i ])
            trueth += list([int(j) for i in tags_tensors for j in i ])

    return trueth, pred



In [39]:
%time train_model_ATE(train_loader, 3)  #Tree epoch

epoch: 0  batch: 1 / 721  loss: 0.9688948392868042  hr: 0  min: 41  sec: 24
epoch: 0  batch: 2 / 721  loss: 0.8510468900203705  hr: 0  min: 33  sec: 40
epoch: 0  batch: 3 / 721  loss: 0.7461363077163696  hr: 0  min: 32  sec: 57
epoch: 0  batch: 4 / 721  loss: 0.7034807950258255  hr: 0  min: 31  sec: 27
epoch: 0  batch: 5 / 721  loss: 0.6527902364730835  hr: 0  min: 30  sec: 27
epoch: 0  batch: 6 / 721  loss: 0.5977026174465815  hr: 0  min: 30  sec: 14
epoch: 0  batch: 7 / 721  loss: 0.6084430856364114  hr: 0  min: 29  sec: 30
epoch: 0  batch: 8 / 721  loss: 0.5584185644984245  hr: 0  min: 30  sec: 7
epoch: 0  batch: 9 / 721  loss: 0.5282848808500502  hr: 0  min: 30  sec: 8
epoch: 0  batch: 10 / 721  loss: 0.5164529800415039  hr: 0  min: 29  sec: 43
epoch: 0  batch: 11 / 721  loss: 0.5134764422069896  hr: 0  min: 29  sec: 25
epoch: 0  batch: 12 / 721  loss: 0.49250348657369614  hr: 0  min: 29  sec: 51
epoch: 0  batch: 13 / 721  loss: 0.4958679423882411  hr: 0  min: 29  sec: 24
epoch: 0 

In [25]:
model_ATE = load_model(model_ATE, 'bert_ATE.pkl')

C:\Users\emink\AppData\Local\Temp\ipykernel_23364\1419488891.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path), strict=False)


In [41]:
%time x, y = test_model_ATE(test_loader)
print(classification_report(x, y, target_names=[str(i) for i in range(3)]))

CPU times: total: 25.1 s
Wall time: 34.9 s
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     64444
           1       0.88      0.85      0.87      4022
           2       0.84      0.76      0.79      2141

    accuracy                           0.98     70607
   macro avg       0.90      0.87      0.88     70607
weighted avg       0.98      0.98      0.98     70607



In [26]:
def test_sentence_ATE(sentence, tokenizer, model_ATE, device=DEVICE):
    """
    Test the ATE model with a single sentence and print the predictions.

    Args:
        sentence (str): The input sentence.
        tokenizer: The tokenizer used during training.
        model_ATE: The trained ATE model.
        device: The device to run the model on ('cpu' or 'cuda').

    Returns:
        List of predicted tags for the input sentence.
    """
    model_ATE.eval()  # Set the model to evaluation mode

    # Tokenize the input sentence
    tokens = tokenizer.tokenize(sentence)
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids_tensor = torch.tensor([input_ids]).to(device)
    attention_mask = torch.tensor([[1] * len(input_ids)]).to(device)

    # Make predictions
    with torch.no_grad():
        outputs = model_ATE(ids_tensors=input_ids_tensor, tags_tensors=None, masks_tensors=attention_mask)
        _, predictions = torch.max(outputs, dim=2)

    # Convert predictions to tags
    predicted_tags = predictions.squeeze().tolist()
    token_tag_pairs = list(zip(tokens, predicted_tags))

    print("Sentence:", sentence)
    print("Predictions:")
    for token, tag in token_tag_pairs:
        print(f"Token: {token}, Predicted Tag: {tag}")

    return token_tag_pairs


In [27]:
sentence = "The food was amazing but the service was terrible."
token_tag_pairs = test_sentence_ATE(sentence, tokenizer, model_ATE)

sentence = "The pasta was delicious, but the service was slow."
token_tag_pairs= test_sentence_ATE(sentence,tokenizer,model_ATE)

sentence = "The ambiance was fantastic, but the food was overpriced."
token_tag_pairs= test_sentence_ATE(sentence,tokenizer,model_ATE)

sentence = "The waiter was very friendly, and the desserts were outstanding"
token_tag_pairs= test_sentence_ATE(sentence,tokenizer,model_ATE)

Sentence: The food was amazing but the service was terrible.
Predictions:
Token: the, Predicted Tag: 0
Token: food, Predicted Tag: 1
Token: was, Predicted Tag: 0
Token: amazing, Predicted Tag: 0
Token: but, Predicted Tag: 0
Token: the, Predicted Tag: 0
Token: service, Predicted Tag: 1
Token: was, Predicted Tag: 0
Token: terrible, Predicted Tag: 0
Token: ., Predicted Tag: 0
Sentence: The pasta was delicious, but the service was slow.
Predictions:
Token: the, Predicted Tag: 0
Token: pasta, Predicted Tag: 1
Token: was, Predicted Tag: 0
Token: delicious, Predicted Tag: 0
Token: ,, Predicted Tag: 0
Token: but, Predicted Tag: 0
Token: the, Predicted Tag: 0
Token: service, Predicted Tag: 1
Token: was, Predicted Tag: 0
Token: slow, Predicted Tag: 0
Token: ., Predicted Tag: 0
Sentence: The ambiance was fantastic, but the food was overpriced.
Predictions:
Token: the, Predicted Tag: 0
Token: am, Predicted Tag: 1
Token: ##bian, Predicted Tag: 1
Token: ##ce, Predicted Tag: 1
Token: was, Predicted T

# Aspect Based Sentiment Analysis

In [28]:

# restaurants_train_ds = dataset_ABSA(pd.read_csv("data/restaurants_train.csv"), tokenizer)
restaurants_train_ds = dataset_ABSA(pd.read_csv("data/ABSA5restaurants_train.csv"), tokenizer)
restaurants_test_ds = dataset_ABSA(pd.read_csv("data/restaurants_test.csv"), tokenizer)
restaurants_val_ds = dataset_ABSA(pd.read_csv("data/V1restaurants_test.csv"), tokenizer)


# restaurants_train_ds = dataset_ABSA(pd.read_csv("data/train_test.csv"), tokenizer)
# restaurants_test_ds = dataset_ABSA(pd.read_csv("data/test_test.csv"), tokenizer)


In [29]:
w,x,y,z = restaurants_train_ds.__getitem__(121)
print(w)
print(len(w))
print(x)
print(len(x))
print(y)
print(len(y))
print(z)

['[cls]', 'i', 'went', 'there', 'in', 'late', 'afternoon', 'for', 'some', 'bite', 'size', 'food', 'and', 'ref', '##les', '##hm', '##ent', 'with', 'my', 'date', '.', '[sep]', 'food']
23
tensor([  100,  1045,  2253,  2045,  1999,  2397,  5027,  2005,  2070,  6805,
         2946,  2833,  1998, 25416,  4244, 14227,  4765,  2007,  2026,  3058,
         1012,   100,  2833])
23
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
23
tensor(1)


In [30]:
def create_mini_batch2(samples):
    # Find the max length in the current batch
    max_len = max([len(s[1]) for s in samples])

    ids_tensors = [torch.cat([s[1], torch.zeros(max_len - len(s[1]), dtype=torch.long)]) for s in samples]
    ids_tensors = torch.stack(ids_tensors)

    segments_tensors = [torch.cat([s[2], torch.zeros(max_len - len(s[2]), dtype=torch.long)]) for s in samples]
    segments_tensors = torch.stack(segments_tensors)

    label_ids = torch.stack([s[3] for s in samples])
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1)

    return ids_tensors, segments_tensors, masks_tensors, label_ids


In [31]:
# train_ds = ConcatDataset([laptops_train_ds, restaurants_train_ds, twitter_train_ds])
# test_ds = ConcatDataset([laptops_test_ds, restaurants_test_ds, twitter_test_ds])

# train_ds = restaurants_train_ds
# test_ds = restaurants_test_ds


# train_loader = DataLoader(train_ds, batch_size=4, collate_fn=create_mini_batch2, shuffle = True)
# test_loader = DataLoader(test_ds, batch_size=50, collate_fn=create_mini_batch2, shuffle = True)

from torch.utils.data import random_split

# Split the training dataset into training and validation sets


# test_size = int(0.5 * len(restaurants_test_ds))
# val_size = len(restaurants_test_ds) - test_size
# test_ds, validation_ds = random_split(restaurants_test_ds, [test_size, val_size])

train_ds = restaurants_train_ds
test_ds = restaurants_test_ds
validation_ds = restaurants_val_ds

# Create DataLoaders
train_loader = DataLoader(train_ds, batch_size=16, collate_fn=create_mini_batch2, shuffle=True)
validation_loader = DataLoader(validation_ds, batch_size=32, collate_fn=create_mini_batch2, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=50, collate_fn=create_mini_batch2, shuffle=True)


In [11]:
# for batch in train_loader:
#     w,x,y,z = batch
#     print(w)
#     print(w.size())
#     print(x)
#     print(x.size())
#     print(y)
#     print(y.size())
#     print(z)
#     print(z.size())
#     break

In [32]:

from torch.optim.lr_scheduler import StepLR
# def train_model_ABSA(loader, epochs):
#     all_data = len(loader)
#     for epoch in range(epochs):
#         finish_data = 0
#         losses = []
#         current_times = []
#         correct_predictions = 0
        
#         for data in loader:
#             t0 = time.time()
#             ids_tensors, segments_tensors, masks_tensors, label_ids = data
#             ids_tensors = ids_tensors.to(DEVICE)
#             segments_tensors = segments_tensors.to(DEVICE)
#             label_ids = label_ids.to(DEVICE)
#             masks_tensors = masks_tensors.to(DEVICE)

#             loss = model_ABSA(ids_tensors=ids_tensors, lable_tensors=label_ids, masks_tensors=masks_tensors, segments_tensors=segments_tensors)
#             losses.append(loss.item())
#             loss.backward()
#             optimizer_ABSA.step()
#             optimizer_ABSA.zero_grad()

#             finish_data += 1
#             current_times.append(round(time.time()-t0,3))
#             current = np.mean(current_times)
#             hr, min, sec = evl_time(current*(all_data-finish_data) + current*all_data*(epochs-epoch-1))
#             print('epoch:', epoch, " batch:", finish_data, "/" , all_data, " loss:", np.mean(losses), " hr:", hr, " min:", min," sec:", sec)         

#         save_model(model_ABSA, 'bert_ABSA3.pkl')
#############################################################################
from torch.optim.lr_scheduler import StepLR
from transformers import get_linear_schedule_with_warmup

def validate_model_ABSA(loader):
    model_ABSA.eval()  # Set the model to evaluation mode
    total_loss = 0
    with torch.no_grad():
        for data in loader:
            ids_tensors, segments_tensors, masks_tensors, label_ids = data
            ids_tensors = ids_tensors.to(DEVICE)
            segments_tensors = segments_tensors.to(DEVICE)
            label_ids = label_ids.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            # Compute the loss during validation
            loss = model_ABSA(ids_tensors=ids_tensors, lable_tensors=label_ids, masks_tensors=masks_tensors, segments_tensors=segments_tensors)
            total_loss += loss.item()

    # Return the average validation loss
    return total_loss / len(loader)


def train_model_ABSA(loader, validation_loader, epochs):

    total_steps = len(loader) * epochs
    warmup_steps = int(0.1 * total_steps)  # 10% warmup

    # Create the scheduler
    scheduler = get_linear_schedule_with_warmup(
        optimizer_ABSA,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )

    all_data = len(loader)
    best_loss = float('inf')
    patience = 10
    early_stop_counter = 0

    # Scheduler
    #scheduler = StepLR(optimizer_ABSA, step_size=1, gamma=0.9)

    for epoch in range(epochs):
        finish_data = 0
        losses = []
        current_times = []
        
        model_ABSA.train()  # Ensure model is in training mode
        for data in loader:
            t0 = time.time()
            ids_tensors, segments_tensors, masks_tensors, label_ids = data
            ids_tensors = ids_tensors.to(DEVICE)
            segments_tensors = segments_tensors.to(DEVICE)
            label_ids = label_ids.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            loss = model_ABSA(ids_tensors=ids_tensors, lable_tensors=label_ids, masks_tensors=masks_tensors, segments_tensors=segments_tensors)
            losses.append(loss.item())
            loss.backward()

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model_ABSA.parameters(), max_norm=1.0)

            optimizer_ABSA.step()

            scheduler.step() #get_linear_schedule_with_warmup

            optimizer_ABSA.zero_grad()

            finish_data += 1
            current_times.append(round(time.time() - t0, 3))
            current = np.mean(current_times)
            hr, min, sec = evl_time(current * (all_data - finish_data) + current * all_data * (epochs - epoch - 1))
            print('epoch:', epoch, " batch:", finish_data, "/", all_data, " loss:", np.mean(losses), " hr:", hr, " min:", min, " sec:", sec)

            # Periodic validation within the epoch
            if finish_data % 100 == 0:
                val_loss = validate_model_ABSA(validation_loader)
                print(f"Validation Loss after {finish_data} batches: {val_loss}")

        # Print learning rate
        for param_group in optimizer_ABSA.param_groups:
            print(f"Learning rate: {param_group['lr']}")

        # Evaluate on validation set
        val_loss = validate_model_ABSA(validation_loader)
        print(val_loss)
        if val_loss < best_loss:
            best_loss = val_loss
            early_stop_counter = 0
            save_model(model_ABSA, 'bert_ABSA8.pkl')
            print("Saving...")
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print("Early stopping!")
                break

        # Step scheduler
        #sscheduler.step()



def test_model_ABSA(loader):
    pred = []
    trueth = []
    with torch.no_grad():
        for data in loader:

            ids_tensors, segments_tensors, masks_tensors, label_ids = data
            ids_tensors = ids_tensors.to(DEVICE)
            segments_tensors = segments_tensors.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            outputs = model_ABSA(ids_tensors, None, masks_tensors=masks_tensors, segments_tensors=segments_tensors)
            
            _, predictions = torch.max(outputs, dim=1)

            pred += list([int(i) for i in predictions])
            trueth += list([int(i) for i in label_ids])

    return trueth, pred



In [51]:
%time train_model_ABSA(train_loader, validation_loader, epochs=10)

epoch: 0  batch: 1 / 122  loss: 1.09127676486969  hr: 0  min: 46  sec: 58
epoch: 0  batch: 2 / 122  loss: 1.1679109334945679  hr: 0  min: 38  sec: 22
epoch: 0  batch: 3 / 122  loss: 1.147869348526001  hr: 0  min: 37  sec: 59
epoch: 0  batch: 4 / 122  loss: 1.1286798119544983  hr: 0  min: 40  sec: 17
epoch: 0  batch: 5 / 122  loss: 1.1413341999053954  hr: 0  min: 45  sec: 30
epoch: 0  batch: 6 / 122  loss: 1.131715714931488  hr: 0  min: 45  sec: 12
epoch: 0  batch: 7 / 122  loss: 1.1377011707850866  hr: 0  min: 44  sec: 8
epoch: 0  batch: 8 / 122  loss: 1.1354601085186005  hr: 0  min: 43  sec: 40
epoch: 0  batch: 9 / 122  loss: 1.133635891808404  hr: 0  min: 44  sec: 44
epoch: 0  batch: 10 / 122  loss: 1.1357030868530273  hr: 0  min: 42  sec: 47
epoch: 0  batch: 11 / 122  loss: 1.1381762569600886  hr: 0  min: 42  sec: 5
epoch: 0  batch: 12 / 122  loss: 1.1279943585395813  hr: 0  min: 41  sec: 15
epoch: 0  batch: 13 / 122  loss: 1.1256739634733934  hr: 0  min: 40  sec: 41
epoch: 0  batch

In [63]:
model_ABSA = load_model(model_ABSA, 'bert_ABSA4.pkl')

C:\Users\emink\AppData\Local\Temp\ipykernel_23364\1419488891.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path), strict=False)


In [64]:
%time x, y = test_model_ABSA(test_loader)
print(classification_report(x, y, target_names=[str(i) for i in range(3)]))

CPU times: total: 4min 55s
Wall time: 2min 16s
              precision    recall  f1-score   support

           0       0.70      0.73      0.72       196
           1       0.53      0.21      0.30       196
           2       0.81      0.94      0.87       727

    accuracy                           0.77      1119
   macro avg       0.68      0.63      0.63      1119
weighted avg       0.74      0.77      0.74      1119



# ATE + ABSA

In [65]:
def predict_model_ABSA(sentence, aspect, tokenizer):
    t1 = tokenizer.tokenize(sentence)
    t2 = tokenizer.tokenize(aspect)

    word_pieces = ['[cls]']
    word_pieces += t1
    word_pieces += ['[sep]']
    word_pieces += t2

    segment_tensor = [0] + [0]*len(t1) + [0] + [1]*len(t2)

    ids = tokenizer.convert_tokens_to_ids(word_pieces)
    input_tensor = torch.tensor([ids]).to(DEVICE)
    segment_tensor = torch.tensor(segment_tensor).to(DEVICE)

    print(f"Tokens: {word_pieces}")
    print(f"Segment Tensor: {segment_tensor}")

    with torch.no_grad():
        outputs = model_ABSA(input_tensor, None, None, segments_tensors=segment_tensor)
        _, predictions = torch.max(outputs, dim=1)
    
    return word_pieces, predictions, outputs
#######################################################
def predict_model_ATE(sentence, tokenizer):
    word_pieces = []
    tokens = tokenizer.tokenize(sentence)
    word_pieces += tokens

    ids = tokenizer.convert_tokens_to_ids(word_pieces)
    input_tensor = torch.tensor([ids]).to(DEVICE)

    with torch.no_grad():
        outputs = model_ATE(input_tensor, None, None)
        _, predictions = torch.max(outputs, dim=2)
    predictions = predictions[0].tolist()

    return word_pieces, predictions, outputs
#############################################################
# def ATE_ABSA(text):
#     terms = []
#     word = ""
    
#     # ATE
#     x, y, _ = predict_model_ATE(text, tokenizer)
#     for i in range(len(y)):
#         if y[i] == 1:  # Start of a new term
#             if len(word) != 0:
#                 terms.append(word.replace(" ##", ""))
#             word = x[i]
#         elif y[i] == 2:  # Continuation of the term
#             if x[i].startswith("##"):
#                 word += x[i][2:]  # Remove the '##' prefix
#             else:
#                 word += " " + x[i]
#         else:
#             if len(word) != 0:
#                 terms.append(word.replace(" ##", ""))
#                 word = ""
#     if len(word) != 0:
#         terms.append(word.replace(" ##", ""))  # Add the last term
    
#     print("tokens:", x)
#     print("ATE:", terms)

#     # ABSA
#     if len(terms) != 0:
#         for term in terms:
#             _, sentiment_class, sentiment_logits = predict_model_ABSA(text, term, tokenizer)

####################################################################################
# def ATE_ABSA(text):
#     terms = []
#     current_term = ""
    
#     # ATE: Aspect Term Extraction
#     tokens, predictions, _ = predict_model_ATE(text, tokenizer)
#     for token, tag in zip(tokens, predictions):
#         if tag == 1:  # Start of a new term
#             if current_term:  # Append the previous term if it exists
#                 terms.append(current_term.strip())  # Finalize the previous term
#             current_term = token  # Start a new term
#         elif tag == 2:  # Continuation of the term
#             if current_term:
#                 if token.startswith("##"):
#                     current_term += token[2:]  # Append subword without '##'
#                 else:
#                     current_term += " " + token
#             else:
#                 # Handle unexpected cases where continuation appears without a start
#                 current_term = token
#         else:  # Tag is 0 (outside)
#             if current_term:  # Append the previous term if it exists
#                 terms.append(current_term.strip())
#                 current_term = ""
#     if current_term:
#         terms.append(current_term.strip())  # Add the last term

#     # **Handle Incorrect Subword Splits**
#     terms = ["".join(term.split(" ##")) for term in terms]

#     print("Tokens:", tokens)
#     print("Extracted Aspect Terms (ATE):", terms)

#     # ABSA: Aspect-Based Sentiment Analysis
#     results = []
#     if terms:
#         for term in terms:
#             _, sentiment_class, sentiment_logits = predict_model_ABSA(text, term, tokenizer)
#             results.append({
#                 "aspect_term": term,
#                 "sentiment_class": sentiment_class.item(),
#                 "logits": sentiment_logits.tolist()
#             })
    
#     # Display Results
#     print("\nAspect Terms and Sentiments (ABSA):")
#     for res in results:
#         print(f"Aspect: {res['aspect_term']}, Sentiment Class: {res['sentiment_class']}, Logits: {res['logits']}")

#     return results

#############################################################################################################################





- NEW PREDICTİON CLASS

In [128]:
import torch

##############################################
# 1) PREDICT_MODEL_ABSA (no attention mask)
##############################################
def predict_model_ABSA(sentence, aspect, tokenizer):
    """
    Predict sentiment class for a given sentence and an extracted aspect term.
    We do NOT add an attention mask, per your request.
    """

    # 1) Tokenize
    sentence_tokens = tokenizer.tokenize(sentence)
    aspect_tokens   = tokenizer.tokenize(aspect)

    # 2) Construct input with two [SEP] tokens:
    #    [CLS] + sentence_tokens + [SEP] + aspect_tokens + [SEP]
    word_pieces = (
        ['[CLS]'] 
        + sentence_tokens 
        + ['[SEP]'] 
        + aspect_tokens 
        + ['[SEP]']
    )
    # Segment IDs:
    #   - 0 for everything up to the first [SEP]
    #   - 1 for aspect tokens + the second [SEP]
    segment_ids = (
        [0]*(1 + len(sentence_tokens) + 1)
        + [1]*(len(aspect_tokens) + 1)
    )

    # 3) Convert to tensors
    input_ids = tokenizer.convert_tokens_to_ids(word_pieces)
    input_ids_tensor   = torch.tensor([input_ids]).to(DEVICE)
    segment_ids_tensor = torch.tensor([segment_ids]).to(DEVICE)

    # Debug prints (optional)
    print("[ABSA Debug] word_pieces:", word_pieces)
    print("[ABSA Debug] segment_ids:", segment_ids)

    # 4) Forward pass (no attention mask)
    with torch.no_grad():
        outputs = model_ABSA(
            ids_tensors      = input_ids_tensor,
            lable_tensors    = None,
            masks_tensors    = None,       # Not adding attention mask
            segments_tensors = segment_ids_tensor
        )
        # outputs shape: [batch_size, 3]
        _, predictions = torch.max(outputs, dim=1)

    # Print raw logits for debugging
    print("[ABSA Debug] logits:", outputs.cpu().numpy())

    return word_pieces, predictions, outputs


#############################################
# 2) PREDICT_MODEL_ATE (no attention mask)
#############################################
def predict_model_ATE(sentence, tokenizer):
    """
    Predict aspect tags (0,1,2) for each token in the given sentence.
    We do NOT add an attention mask, per your request.
    """

    # 1) Tokenize the sentence
    tokens = tokenizer.tokenize(sentence)

    # 2) Convert tokens to IDs
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids_tensor = torch.tensor([input_ids]).to(DEVICE)

    # 3) Forward pass (no attention mask)
    with torch.no_grad():
        outputs = model_ATE(
            ids_tensors   = input_ids_tensor,
            tags_tensors  = None,  # no labels at inference
            masks_tensors = None   # skipping attention mask
        )
        # outputs shape: [batch_size, seq_len, 3]
        # we do a max across dim=2 to get predicted tag
        _, predictions = torch.max(outputs, dim=2)

    # predictions is [1, seq_len], so get the first row
    predictions = predictions[0].cpu().tolist()

    return tokens, predictions, outputs


###########################################
# 3) ATE_ABSA: Extract aspects, then ABSA
###########################################
def ATE_ABSA(text):
    """
    1) Extract aspect terms (ATE) from the text using predict_model_ATE.
    2) For each extracted aspect, call predict_model_ABSA for sentiment on the
       entire sentence plus that single aspect.
    3) Print final results in a user-friendly format:
       SENTENCE: ...
       - Aspect 'foo': Sentiment = ...
       - Aspect 'bar': Sentiment = ...
    """

    # Ensure both models are in eval mode for inference
    model_ATE.eval()
    model_ABSA.eval()

    # -----------------------------
    # A) Aspect Term Extraction
    # -----------------------------
    aspect_terms = []
    current_term = ""

    # 1) Predict tokens & tags from ATE
    tokens, predicted_tags, _ = predict_model_ATE(text, tokenizer)

    # 2) Merge subwords into aspect terms
    for token, tag in zip(tokens, predicted_tags):
        if tag == 1:  # B-term
            if current_term:
                aspect_terms.append(current_term.strip())
            current_term = token
        elif tag == 2:  # I-term
            if current_term:
                if token.startswith("##"):
                    current_term += token[2:]
                else:
                    current_term += " " + token
            else:
                # If we get '2' without a preceding '1'
                current_term = token
        else:  # tag == 0 => outside
            if current_term:
                aspect_terms.append(current_term.strip())
                current_term = ""

    # If leftover aspect term
    if current_term:
        aspect_terms.append(current_term.strip())

    # 3) Clean leftover '##'
    aspect_terms = [term.replace(" ##", "") for term in aspect_terms]

    # Debug: Print ATE results
    print("[ATE Debug] tokens:", tokens)
    print("[ATE Debug] predicted_tags:", predicted_tags)
    print("[ATE Debug] aspect_terms:", aspect_terms)

    # -----------------------------
    # B) Aspect-Based Sentiment
    # -----------------------------
    results = []

    # For each aspect, we do ABSA with full sentence + single aspect
    for term in aspect_terms:
        word_pieces, pred_class, logits = predict_model_ABSA(text, term, tokenizer)

        # Convert logits to Python list for logging
        logits_list = logits.squeeze().cpu().numpy().tolist()
        
        results.append({
            "aspect_term": term,
            "sentiment_class": pred_class.item(),  # e.g., 0=neg, 1=neu, 2=pos
            "logits": logits_list
        })

    # -----------------------------
    # Print final results
    # -----------------------------
    print(f"\nSENTENCE: {text}")
    for r in results:
        aspect   = r["aspect_term"]
        polarity = r["sentiment_class"]
        print(f" - Aspect '{aspect}': Sentiment = {polarity}")

    return results


In [129]:
model_ABSA = load_model(model_ABSA, 'bert_ABSA4.pkl')
model_ATE = load_model(model_ATE, 'bert_ATE.pkl')

C:\Users\emink\AppData\Local\Temp\ipykernel_23364\1419488891.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path), strict=False)


- Code to randomly slect and test sentences from testing.csv

In [141]:
import pandas as pd
import random

def test_random_sentences(
    csv_path="testing.csv", 
    num_samples=5
):
    """
    1) Reads 'testing.csv' which has:
         Tokens (full sentence), 
         Tags (aspect), 
         Polarities (gold sentiment).
    2) Groups by 'Tokens' so each unique sentence is one group.
    3) Randomly picks 'num_samples' distinct sentences from that CSV.
    4) For each sentence, calls your ATE_ABSA pipeline.
    5) Prints a side-by-side comparison of:
         - Aspects & Sentiment (model vs. CSV).
    """

    # -------------------------------
    # A) LOAD & GROUP THE CSV
    # -------------------------------
    df = pd.read_csv(csv_path)

    # Group by 'Tokens' => each group is effectively a distinct full sentence
    grouped = df.groupby("Tokens")

    # Extract all distinct sentences from the CSV
    all_sentences = list(grouped.groups.keys())

    # If 'num_samples' is bigger than total distinct sentences, clamp it
    num_samples = min(num_samples, len(all_sentences))

    # Randomly pick 'num_samples' distinct sentences
    chosen_sentences = random.sample(all_sentences, k=num_samples)

    # -------------------------------
    # B) FOR EACH CHOSEN SENTENCE
    # -------------------------------
    for sentence in chosen_sentences:
        # 1) Lookup all aspects from the CSV for this sentence
        sub_df = grouped.get_group(sentence)
        # e.g. might have:
        #   "But the staff was so horrible to us."  staff     negative
        #   "But the staff was so horrible to us."  food      positive
        # etc.

        # Convert them to a list of (aspect, gold_polarity)
        gold_aspects = []
        for _, row in sub_df.iterrows():
            gold_aspects.append((row["Tags"], row["Polarities"]))

        # 2) Call your ATE_ABSA pipeline => model aspects & sentiment
        #    This function is what you already wrote or the improved version.
        #    Must be defined in your code. For example:
        model_results = ATE_ABSA(sentence)
        # model_results is a list of dicts like:
        # [
        #   { "aspect_term": "staff", "sentiment_class": 0_or_1_or_2, "logits": [...] },
        #   ...
        # ]

        # Convert the model's numeric polarity to text to compare easily
        # For example, you might do:
        def polarity_to_text(idx):
            # Example mapping (adapt if your model uses different labels)
            # 0=negative, 1=neutral, 2=positive
            if idx == 0:
                return "negative"
            elif idx == 1:
                return "neutral"
            elif idx == 2:
                return "positive"
            return "???"

        # Build a dict from aspect -> predicted_polarity
        # (If model finds multiple aspects, might have duplicates)
        model_aspects_dict = {}
        for r in model_results:
            aspect_term = r["aspect_term"]
            polarity_idx = r["sentiment_class"]
            polarity_text = polarity_to_text(polarity_idx)
            model_aspects_dict[aspect_term] = polarity_text

        # 3) Print a side-by-side summary
        print("==========================================================")
        print(f"SENTENCE: {sentence}")
        print("\n[CSV file says aspects:]")
        for aspect, gold_pol in gold_aspects:
            print(f"  - Aspect '{aspect}' => Gold Polarity: {gold_pol}")

        print("\n[MODEL found aspects:]")
        if len(model_results) == 0:
            print("  - (No aspects found)")
        else:
            for r in model_results:
                # e.g. aspect='staff', pol=2 => 'positive'
                aspect_term = r["aspect_term"]
                predicted_pol = polarity_to_text(r["sentiment_class"])
                print(f"  - Aspect '{aspect_term}' => Predicted Polarity: {predicted_pol}")

        print("==========================================================\n")

    # Done. We just tested 'num_samples' random sentences from 'testing.csv'.


In [143]:
test_random_sentences(csv_path="testing.csv", num_samples=5)


[ATE Debug] tokens: ['it', 'took', 'them', '15', 'minutes', 'to', 'put', 'water', 'in', 'our', 'glasses', '.']
[ATE Debug] predicted_tags: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[ATE Debug] aspect_terms: ['water']
[ABSA Debug] word_pieces: ['[CLS]', 'it', 'took', 'them', '15', 'minutes', 'to', 'put', 'water', 'in', 'our', 'glasses', '.', '[SEP]', 'water', '[SEP]']
[ABSA Debug] segment_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[ABSA Debug] logits: [[-1.0276011   0.20085283  1.0429422 ]]

SENTENCE: It took them 15 minutes to put water in our glasses.
 - Aspect 'water': Sentiment = 2
SENTENCE: It took them 15 minutes to put water in our glasses.

[CSV file says aspects:]
  - Aspect 'water' => Gold Polarity: neutral

[MODEL found aspects:]
  - Aspect 'water' => Predicted Polarity: positive

[ATE Debug] tokens: ['as', 'soon', 'as', 'my', 'father', 'lifted', 'his', 'pen', 'from', 'the', 'check', 'a', 'chef', 'appeared', 'to', 'usher', 'us', 'out', '.']
[ATE Debug] predicted_tags: [

In [142]:
text = "the service was discusting."
ATE_ABSA(text)

[ATE Debug] tokens: ['the', 'service', 'was', 'discus', '##ting', '.']
[ATE Debug] predicted_tags: [0, 1, 0, 0, 0, 0]
[ATE Debug] aspect_terms: ['service']
[ABSA Debug] word_pieces: ['[CLS]', 'the', 'service', 'was', 'discus', '##ting', '.', '[SEP]', 'service', '[SEP]']
[ABSA Debug] segment_ids: [0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[ABSA Debug] logits: [[ 0.35645038  0.8861331  -1.1608526 ]]

SENTENCE: the service was discusting.
 - Aspect 'service': Sentiment = 1


[{'aspect_term': 'service',
  'sentiment_class': 1,
  'logits': [0.3564503788948059, 0.886133074760437, -1.1608525514602661]}]

In [131]:
text = "Staff was exellent."
ATE_ABSA(text)

[ATE Debug] tokens: ['staff', 'was', 'ex', '##elle', '##nt', '.']
[ATE Debug] predicted_tags: [1, 0, 0, 0, 0, 0]
[ATE Debug] aspect_terms: ['staff']
[ABSA Debug] word_pieces: ['[CLS]', 'staff', 'was', 'ex', '##elle', '##nt', '.', '[SEP]', 'staff', '[SEP]']
[ABSA Debug] segment_ids: [0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[ABSA Debug] logits: [[-1.0926075   0.17564873  1.1215113 ]]

SENTENCE: Staff was exellent.
 - Aspect 'staff': Sentiment = 2


[{'aspect_term': 'staff',
  'sentiment_class': 2,
  'logits': [-1.0926074981689453, 0.1756487339735031, 1.1215113401412964]}]

In [132]:
text = "The pasta was delicious, but the service was slow."
ATE_ABSA(text)

[ATE Debug] tokens: ['the', 'pasta', 'was', 'delicious', ',', 'but', 'the', 'service', 'was', 'slow', '.']
[ATE Debug] predicted_tags: [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[ATE Debug] aspect_terms: ['pasta', 'service']
[ABSA Debug] word_pieces: ['[CLS]', 'the', 'pasta', 'was', 'delicious', ',', 'but', 'the', 'service', 'was', 'slow', '.', '[SEP]', 'pasta', '[SEP]']
[ABSA Debug] segment_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[ABSA Debug] logits: [[-1.0911393  0.1442051  1.1300265]]
[ABSA Debug] word_pieces: ['[CLS]', 'the', 'pasta', 'was', 'delicious', ',', 'but', 'the', 'service', 'was', 'slow', '.', '[SEP]', 'service', '[SEP]']
[ABSA Debug] segment_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[ABSA Debug] logits: [[-1.1000597   0.14454973  1.1359445 ]]

SENTENCE: The pasta was delicious, but the service was slow.
 - Aspect 'pasta': Sentiment = 2
 - Aspect 'service': Sentiment = 2


[{'aspect_term': 'pasta',
  'sentiment_class': 2,
  'logits': [-1.091139316558838, 0.14420509338378906, 1.1300264596939087]},
 {'aspect_term': 'service',
  'sentiment_class': 2,
  'logits': [-1.1000597476959229, 0.14454972743988037, 1.1359444856643677]}]

In [133]:
text = "The ambiance was fantastic, but the food was overpriced."
ATE_ABSA(text)

[ATE Debug] tokens: ['the', 'am', '##bian', '##ce', 'was', 'fantastic', ',', 'but', 'the', 'food', 'was', 'over', '##pr', '##ice', '##d', '.']
[ATE Debug] predicted_tags: [0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[ATE Debug] aspect_terms: ['am', '##bian', '##ce', 'food']
[ABSA Debug] word_pieces: ['[CLS]', 'the', 'am', '##bian', '##ce', 'was', 'fantastic', ',', 'but', 'the', 'food', 'was', 'over', '##pr', '##ice', '##d', '.', '[SEP]', 'am', '[SEP]']
[ABSA Debug] segment_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[ABSA Debug] logits: [[-1.0807803   0.19206576  1.1046661 ]]
[ABSA Debug] word_pieces: ['[CLS]', 'the', 'am', '##bian', '##ce', 'was', 'fantastic', ',', 'but', 'the', 'food', 'was', 'over', '##pr', '##ice', '##d', '.', '[SEP]', '#', '#', 'bi', '##an', '[SEP]']
[ABSA Debug] segment_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
[ABSA Debug] logits: [[-1.0797855   0.18983082  1.1042734 ]]
[ABSA Debug] word_pieces: ['[CLS]'

[{'aspect_term': 'am',
  'sentiment_class': 2,
  'logits': [-1.080780267715454, 0.1920657604932785, 1.1046661138534546]},
 {'aspect_term': '##bian',
  'sentiment_class': 2,
  'logits': [-1.0797854661941528, 0.18983082473278046, 1.1042734384536743]},
 {'aspect_term': '##ce',
  'sentiment_class': 2,
  'logits': [-1.0788630247116089, 0.1877373307943344, 1.1043490171432495]},
 {'aspect_term': 'food',
  'sentiment_class': 2,
  'logits': [-1.0792627334594727, 0.19420789182186127, 1.1026583909988403]}]

In [137]:
text = "The ambiance was terrible, but the food was exellent."
ATE_ABSA(text)

[ATE Debug] tokens: ['the', 'am', '##bian', '##ce', 'was', 'terrible', ',', 'but', 'the', 'food', 'was', 'ex', '##elle', '##nt', '.']
[ATE Debug] predicted_tags: [0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[ATE Debug] aspect_terms: ['am', '##bian', '##ce', 'food']
[ABSA Debug] word_pieces: ['[CLS]', 'the', 'am', '##bian', '##ce', 'was', 'terrible', ',', 'but', 'the', 'food', 'was', 'ex', '##elle', '##nt', '.', '[SEP]', 'am', '[SEP]']
[ABSA Debug] segment_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[ABSA Debug] logits: [[ 1.9913644 -0.649262  -1.0417598]]
[ABSA Debug] word_pieces: ['[CLS]', 'the', 'am', '##bian', '##ce', 'was', 'terrible', ',', 'but', 'the', 'food', 'was', 'ex', '##elle', '##nt', '.', '[SEP]', '#', '#', 'bi', '##an', '[SEP]']
[ABSA Debug] segment_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
[ABSA Debug] logits: [[ 1.9827173  -0.63632214 -1.0703382 ]]
[ABSA Debug] word_pieces: ['[CLS]', 'the', 'am', '##bian', '##ce', 'was',

[{'aspect_term': 'am',
  'sentiment_class': 0,
  'logits': [1.9913643598556519, -0.649262011051178, -1.0417598485946655]},
 {'aspect_term': '##bian',
  'sentiment_class': 0,
  'logits': [1.9827172756195068, -0.6363221406936646, -1.070338249206543]},
 {'aspect_term': '##ce',
  'sentiment_class': 0,
  'logits': [1.985780954360962, -0.6485776901245117, -1.0585136413574219]},
 {'aspect_term': 'food',
  'sentiment_class': 0,
  'logits': [1.9983842372894287, -0.6649143099784851, -1.0344505310058594]}]

In [138]:
text="The waiter was very friendly, and the desserts were outstanding"
ATE_ABSA(text)

[ATE Debug] tokens: ['the', 'waiter', 'was', 'very', 'friendly', ',', 'and', 'the', 'dessert', '##s', 'were', 'outstanding']
[ATE Debug] predicted_tags: [0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0]
[ATE Debug] aspect_terms: ['waiter', 'dessert', '##s']
[ABSA Debug] word_pieces: ['[CLS]', 'the', 'waiter', 'was', 'very', 'friendly', ',', 'and', 'the', 'dessert', '##s', 'were', 'outstanding', '[SEP]', 'waiter', '[SEP]']
[ABSA Debug] segment_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[ABSA Debug] logits: [[-1.0720174   0.20034967  1.0933613 ]]
[ABSA Debug] word_pieces: ['[CLS]', 'the', 'waiter', 'was', 'very', 'friendly', ',', 'and', 'the', 'dessert', '##s', 'were', 'outstanding', '[SEP]', 'dessert', '[SEP]']
[ABSA Debug] segment_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[ABSA Debug] logits: [[-1.0720706   0.20036568  1.0933958 ]]
[ABSA Debug] word_pieces: ['[CLS]', 'the', 'waiter', 'was', 'very', 'friendly', ',', 'and', 'the', 'dessert', '##s', 'were', 'outstanding', '[SEP]

[{'aspect_term': 'waiter',
  'sentiment_class': 2,
  'logits': [-1.0720174312591553, 0.20034967362880707, 1.093361258506775]},
 {'aspect_term': 'dessert',
  'sentiment_class': 2,
  'logits': [-1.072070598602295, 0.20036567747592926, 1.0933958292007446]},
 {'aspect_term': '##s',
  'sentiment_class': 2,
  'logits': [-1.0723247528076172, 0.20028358697891235, 1.0936135053634644]}]

In [140]:
text="The waiter was very friendly, and the dessert was discusting"
ATE_ABSA(text)

[ATE Debug] tokens: ['the', 'waiter', 'was', 'very', 'friendly', ',', 'and', 'the', 'dessert', 'was', 'discus', '##ting']
[ATE Debug] predicted_tags: [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[ATE Debug] aspect_terms: ['waiter', 'dessert']
[ABSA Debug] word_pieces: ['[CLS]', 'the', 'waiter', 'was', 'very', 'friendly', ',', 'and', 'the', 'dessert', 'was', 'discus', '##ting', '[SEP]', 'waiter', '[SEP]']
[ABSA Debug] segment_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[ABSA Debug] logits: [[-1.0736802  0.1992836  1.095133 ]]
[ABSA Debug] word_pieces: ['[CLS]', 'the', 'waiter', 'was', 'very', 'friendly', ',', 'and', 'the', 'dessert', 'was', 'discus', '##ting', '[SEP]', 'dessert', '[SEP]']
[ABSA Debug] segment_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[ABSA Debug] logits: [[-1.0732858   0.19933473  1.0947171 ]]

SENTENCE: The waiter was very friendly, and the dessert was discusting
 - Aspect 'waiter': Sentiment = 2
 - Aspect 'dessert': Sentiment = 2


[{'aspect_term': 'waiter',
  'sentiment_class': 2,
  'logits': [-1.0736801624298096, 0.19928359985351562, 1.0951329469680786]},
 {'aspect_term': 'dessert',
  'sentiment_class': 2,
  'logits': [-1.0732858180999756, 0.19933472573757172, 1.0947171449661255]}]